Александр Журавлев DST-120  
[ссылка](https://github.com/drSever/drSever_data_science/tree/main/Small_training_projects/project_3) на GitHub проекта.
***
## Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов **petersburg** и **magadan** в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.  
Также ответьте на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.

Исходные данные:

In [1]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

***

## Решение

#### 1). Загрузим необходимые библиотеки.

In [2]:
# pip install scipy
# pip install statsmodels

In [3]:
import pandas as pd

from scipy.stats import shapiro # тест Шапиро-Уилка
from scipy.stats import normaltest # тест Д’Агостино
from scipy.stats import ttest_ind # независимый T-тест

from scipy.stats import pearsonr # корреляция Пирсона


***

#### 2). Создадим новый DataFrame **data_mussels**, объединив 2 массива **petersburg** и **magadan**.

In [4]:
# учитывая разную длинну массивов, объединим их 
# через создание объектов Series

petersburg_series = pd.Series(
    data = petersburg,
    name = 'petersburg'
)

magadan_series = pd.Series(
    data = magadan,
    name = 'magadan'
)

# создадим DataFrame, объединив 2 объекта Series
data_mussels = pd.concat([petersburg_series, magadan_series], axis=1)

In [5]:
data_mussels.head(8)

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


**Вывод.** Мы создали новый DataFrame, объеденив два изначальных массива, однако из-за их разной длины в итоговой таблице имеется пустое значение **NaN**.

***

#### 3). Проверим данные на нормальность.

Сформулируем гипотезы.

In [6]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [7]:
alpha = 0.05

- Используем **тест Шапиро-Уилка**

In [8]:
print('Результаты теста Шапиро-Уилка')

# проводим тест и выводим результат
_, p = shapiro(data_mussels)
print('p=%.3f' % p)

# Интерпретация результатов теста
if p > alpha:
	print(H0)
else:
	print(Ha)

Результаты теста Шапиро-Уилка
p=1.000
Данные распределены нормально


- используем **тест Д’Агостино**.  
К сожалению, в данной ситуации мы не можем использовать данный тест, т.к. в нашем DataFrame имеется пустое значение **NaN** и тест возвращает нам также **NaN**. Если мы попытаемся удалить строку с пустым значением либо проигнорировать его, то получим другую ошибку - количество значений в данных меньше 8.

**Вывод.** Согласно тесту Шапиро-Уилка данные распределены нормально.

***

#### 4). Проверим данные на наличие корреляции.

Так как мы имеем дело с непрерывными признаками и данные распределены нормально, то использовать мы будем **корреляцию Пирсона**.

In [9]:
data_mussels.corr()

,petersburg,magadan
petersburg,1.000000,0.255095
magadan,0.255095,1.000000


Итак, коэффициент корреляции 0,26, что соответствует отсутствию связи или очень слабой связи между признаками.
Однако, давайте решим эту задачу другим способом, получив коэффициент **p-value**.

Сформулируем гипотезы.

In [10]:
H0 = 'Нет зависимости между размером раковины мидий в двух разных местах.'
Ha = 'Есть зависимость между размерами раковины мидий в двух разных местах.'

Уровень значимости **alpha** уже установлен, проведем тест.

In [11]:
# чтобы обойти ошибку с NaN удалим строку его содержащюю
data_mussels_dropna = data_mussels.dropna()

# выполняем тест Пирсона
corr, p = pearsonr(data_mussels_dropna['petersburg'], data_mussels_dropna['magadan'])

# округлим значения
corr = round(corr, 3)
p = round(p, 3)

# выводим результат теста
print('Результаты теста Пирсона')
print(f'corr = {corr}, p-value = {p}')

# интерпретируем результат теста
if p>alpha:
    print(f"{p} > {alpha}. {H0}")
else:
    print(f"{p} <= {alpha}. {Ha}")

Результаты теста Пирсона
corr = 0.255, p-value = 0.581
0.581 > 0.05. Нет зависимости между размером раковины мидий в двух разных местах.


**Вывод.** Отсутствует зависимость между размерами раковин мидий в двух разных местах.

***

#### 5). Какова разница в среднем размере мидии в зависимости от города-производителя.

У нас имеется 1 независимая категориальная переменная **город-производитель** и одна зависимая количественная переменная - **размер мидии**.  
Группы происходят из разных совокупностей (**petersburg**, **magadan**).  
Данные, как мы выяснили, распределены нормально.  
Мы сравниваем 2 группы, объектов в выборке меньше 30, отсутствует информация о разбросе данных.  

Следовательно, мы будем использовать параметрический **независимый Т-тест**.

Сформулируем гипотезы.

In [12]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в двух разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в двух разных местах.'

Уровень значимости **alpha** уже установлен, проведем тест.

In [13]:
# проводим Т-тест, игнорируя NaN, по умолчанию equal_var=True 
_, p = ttest_ind(data_mussels['petersburg'], data_mussels['magadan'], nan_policy='omit')

# округлим значение p-value
p = round(p, 3)

# выводим и интерпретируем результаты теста
if p>alpha:
  print(f"{p} > {alpha}.", f"Мы не можем отвергнуть нулевую гипотезу. {H0}", sep="\n")
else:
  print(f"{p} <= {alpha}.", f"Мы отвергаем нулевую гипотезу. {Ha}", sep="\n")

0.005 <= 0.05.
Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в двух разных местах.


**Вывод.** Есть значимая разница между средним размером раковины мидий в зависимости от города-производителя.